# Pronostico

In [1]:
#Paqueteria a utilizar

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization library
import matplotlib.pyplot as plt # visualization library
#import plotly.plotly as py # visualization library
from plotly.offline import init_notebook_mode, iplot # plotly offline mode
init_notebook_mode(connected=True) 
import plotly.graph_objs as go # plotly graphical object

import chart_studio.plotly as py

import warnings        
# ignore filters
warnings.filterwarnings("ignore") # if there is a warning after some codes, this will avoid us to see them.
plt.style.use('ggplot') # style of plots. ggplot is one of the most used style, I also like it.
# Any results you write to the current directory are saved as output.

Load data

In [ ]:
df = pd.read_csv("/Users/mafer/Documents/.csv")

In [ ]:
df.shape

In [ ]:
df.head()

Data cleaning

In [ ]:
#Borramos columnas con na
df = df[pd.isna(df.column)==False]

In [ ]:
df.shape

In [ ]:
# drop unused features
drop_list = ['columns']

In [ ]:
df.drop(drop_list, axis=1,inplace = True)

In [ ]:
df.info()

Data visualization

In [ ]:
print(df['column'].value_counts())
plt.figure(figsize=(22,10))
sns.countplot(df['column'])
plt.show()

# Time series prediction with ARIMA

In [ ]:
# Mean variable
plt.figure(figsize=(22,10))
plt.plot(variable_bin.Date,variable_bin.variable)
plt.title("Mean variable")
plt.xlabel("Date")
plt.ylabel("Mean variable")
plt.show()

# lets create time series from variable 
timeSeries = variable_bin.loc[:, ["Date","Variable"]]
timeSeries.index = timeSeries.Date
ts = timeSeries.drop("Date",axis=1)

In [ ]:
ts

In [3]:
#Now lets check stationary of time series. We can check stationarity using the following methods:
# -Plotting Rolling Statistics: We have a window lets say window size is 6 and then we find rolling mean and variance to check 
#     stationary.
# -Dickey-Fuller Test: The test results comprise of a Test Statistic and some Critical Values for difference confidence levels. 
#     If the test statistic is less than the critical value, we can say that time series is stationary.

In [ ]:
# adfuller library 
from statsmodels.tsa.stattools import adfuller
# check_adfuller
def check_adfuller(ts):
    # Dickey-Fuller test
    result = adfuller(ts, autolag='AIC')
    print('Test statistic: ' , result[0])
    print('p-value: '  ,result[1])
    print('Critical Values:' ,result[4])
# check_mean_std
def check_mean_std(ts):
    #Rolling statistics
    rolmean = ts.rolling(window=6).mean()
    rolstd = ts.rolling(window=6).std()
    plt.figure(figsize=(22,10))   
    orig = plt.plot(ts, color='red',label='Original')
    mean = plt.plot(rolmean, color='black', label='Rolling Mean')
    std = plt.plot(rolstd, color='green', label = 'Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Variable")
    plt.title('Rolling Mean & Standard Deviation')
    plt.legend()
    plt.show()
    
# check stationary: mean, variance(std)and adfuller test
check_mean_std(ts)
check_adfuller(ts.MeanVariable)

Make a time series stationary? 

In [ ]:
As we mentioned before, there are 2 reasons behind non-stationarity of time series:
-Trend: varying mean over time. We need constant mean for stationary of time series.
-Seasonality: variations at specific time. We need constant variations for stationary of time series.

In [ ]:
First solve trend(constant mean) problem
Most popular method is moving average.
Moving average: We have window that take the average over the past 'n' sample. 'n' is window size.

In [ ]:
# Moving average method
window_size = 6 #6 días atrás 
moving_avg = ts.rolling(6).mean()

In [ ]:
plt.figure(figsize=(22,10))
plt.plot(ts, color = "red",label = "Original")
plt.plot(moving_avg, color='black', label = "moving_avg_mean")
plt.title("Mean Variable")
plt.xlabel("Date")
plt.ylabel("Mean Variable")
plt.legend()
plt.show()

In [ ]:
ts_moving_avg_diff = ts - moving_avg
ts_moving_avg_diff.dropna(inplace=True) # first 6 is nan value due to window size

In [4]:
# check stationary: mean, variance(std)and adfuller test
check_mean_std(ts_moving_avg_diff)
check_adfuller(ts_moving_avg_diff.MeanVariable)

NameError: name 'check_mean_std' is not defined

In [5]:
#Hasta este punto podría ser estacionaria

In [ ]:
Differencing method: It is one of the most common method. Idea is that take difference between time series 
and shifted time series.

In [ ]:
# differencing method
ts_diff = ts - ts.shift()
plt.figure(figsize=(22,10))
plt.plot(ts_diff)
plt.title("Differencing method") 
plt.xlabel("Date")
plt.ylabel("Differencing Mean Variable")
plt.show()

In [ ]:
ts_diff.dropna(inplace=True) # due to shifting there is nan values
# check stationary: mean, variance(std)and adfuller test
check_mean_std(ts_diff)
check_adfuller(ts_diff.MeanVariable)

In [ ]:
Forecasting a Time Series
-Two different methodsthat are moving average and differencing methods to avoid trend and seasonality problem

In [6]:
Prediction method is ARIMA that is Auto-Regressive Integrated Moving Averages.
AR: Auto-Regressive (p): AR terms are just lags of dependent variable. For example lets say p is 3, 
        we will use x(t-1), x(t-2) and x(t-3) to predict x(t)
I: Integrated (d): These are the number of nonseasonal differences. For example, in our case we take the first order 
        difference. So we pass that variable and put d=0
MA: Moving Averages (q): MA terms are lagged forecast errors in prediction equation.

SyntaxError: invalid syntax (<ipython-input-6-7ba98dcad1f4>, line 1)

In [8]:
#Forecasting a time series

The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:

1. Number of AR (Auto-Regressive) terms (p): AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).
2. Number of MA (Moving Average) terms (q): MA terms are lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where e(i) is the difference between the moving average at ith instant and actual value.
3. Number of Differences (d): These are the number of nonseasonal differences, i.e. in this case we took the first order difference. So either we can pass that variable and put d=0 or pass the original variable and put d=1. Both will generate same results.

An importance concern here is how to determine the value of ‘p’ and ‘q’. We use two plots to determine these numbers.

1. Autocorrelation Function (ACF): It is a measure of the correlation between the TS with a lagged version of itself. For instance at lag 5, ACF would compare series at time instant ‘t1’…’t2’ with series at instant ‘t1-5’…’t2-5’ (t1-5 and t2 being end points).
2. Partial Autocorrelation Function (PACF): This measures the correlation between the TS with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons. Eg at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

Veamos también un diagrama de autocorrelación de la serie temporal. Esto también está integrado en Pandas. El siguiente ejemplo traza la autocorrelación para un gran número de retrasos en la serie de tiempo.

In [ ]:
#ACF plot
pd.plotting.autocorrelation_plot(ts_log_diff) #Revisar en que punto/rezago de la grafica la autocorrelacion es significativa 
                                              #(mayor a .5) para fijar ahi el lag value = p
                                              

#plt.figure(figsize=(12, 6))
#pd.plotting.autocorrelation_plot(series)
#plt.show()

In [ ]:
In this plot, the two dotted lines on either sides of 0 are the confidence interevals. These can be used to determine the ‘p’ and ‘q’ values as:

p – The lag value where the PACF chart crosses the upper confidence interval for the first time (p=2).

q – The lag value where the ACF chart crosses the upper confidence interval for the first time(q=2).

Now, lets make 3 different ARIMA models considering individual as well as combined effects. We will also print the RSS for each. Please note that here RSS is for the values of residuals and not actual series.

We need to load the ARIMA model first:

In [9]:
#Necesitamos entender bien ACF y PACF para saber el orden de p,d,q

Defina el modelo llamando a ARIMA () y pasando los parámetros p, d y q. El modelo se prepara sobre los datos de entrenamiento llamando a la función fit(). Se pueden hacer predicciones llamando a la función predict() y especificando el índice de la hora u horas a predecir. Comencemos con algo simple. Ajustaremos un modelo ARIMA a todo el conjunto de datos de Shampoo Sales y revisaremos los errores residuales.

Primero, ajustamos un modelo ARIMA (5,1,0). Esto establece el valor de retraso en 5 para la autorregresión (AR), usa un orden de diferencia de 1 para hacer estacionarias las series de tiempo y usa un modelo de promedio móvil de 0.

Al ajustar el modelo, se proporciona mucha información de depuración sobre el ajuste del modelo de regresión lineal. Podemos desactivar esto estableciendo el argumento disp en 0.

In [10]:
from statsmodels.tsa.arima_model import ARIMA, ARMAResults

AR Model

In [ ]:
model = ARIMA(ts_log, order = (2,1,0)) #p,d,q para los modelos AR solo ocupamos p que en este caso es igual a 2, q=1 por ser la 
                                       #primera diferencia y 0 por ser un modelo AR 
#Fit para entrenar el modelo    
results_AR = model.fit(disp = -1) #Al ajustar el modelo, se proporciona mucha información de depuración sobre el ajuste del 
                                  #modelo de regresión lineal.Podemos desactivar esto estableciendo el argumento disp en 0.

plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color = 'red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues - ts_log_diff)**2)) #Residual sum of squares

MA Model

In [ ]:
model = ARIMA(ts_log, order= (0, 1, 2))  #p,d,q en este caso por ser modelo MA utilizamos los rezagos de error por lo que p=1,
                                        #es un modelo de prime diferencia con q=2 por utilizar dos rezagos en el error
#Fit para entrenar el modelo    
results_MA = model.fit(disp = -1) #Al ajustar el modelo, se proporciona mucha información de depuración sobre el ajuste del 
                                  #modelo de regresión lineal.Podemos desactivar esto estableciendo el argumento disp en 0. 


plt.plot(ts_log_diff)
plt.plot(results_MA.fittedvalues, color= 'red')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues - ts_log_diff)**2))

Combined Model

In [ ]:
model = ARIMA(ts_log, order = (2, 1, 2))  #p,d,q 
#Fit para entrenar el modelo 
results_ARIMA = model.fit(disp = -1)#Al ajustar el modelo, se proporciona mucha información de depuración sobre el ajuste del 
                                  #modelo de regresión lineal.Podemos desactivar esto estableciendo el argumento disp en 0.

plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color= 'red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues - ts_log_diff)**2))

Ejecutando el ejemplo imprime un resumen del modelo ajustado. Esto resume los valores de los coeficientes utilizados, así como la habilidad del ajuste en las observaciones de la muestra.

In [ ]:
model_fit.summary()

Primero, obtenemos un plot de los errores residuales, lo que sugiere que todavía puede haber alguna información de tendencia que el modelo no capture.

In [ ]:
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()

A continuación, obtenemos un plot de densidad de los valores de error residual, lo que sugiere que los errores son gaussianos, pero pueden no estar centrados en cero.

In [ ]:
residuals.plot(kind='kde')
pyplot.show()

Se muestra la distribución de los errores residuales. Los resultados muestran que, de hecho, hay un sesgo en la predicción (una media distinta de cero en los residuos).

In [ ]:
residuals.describe()

Tenga en cuenta que, aunque anteriormente utilizamos todo el conjunto de datos para el análisis de series de tiempo, idealmente realizaríamos este análisis solo en el conjunto de datos de entrenamiento al desarrollar un modelo predictivo.

A continuación, veamos cómo podemos usar el modelo ARIMA para hacer pronósticos.

NOTE: We can split the training dataset into train and test sets, use the train set to fit the model, and generate a prediction for each element on the test set.

Taking it back to original scale

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff.head()

In [ ]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_diff_cumsum.head()

In [ ]:
predictions_ARIMA_log = pd.Series(ts_log.iloc[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(ts)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts)))

Validating Forecasts

In [ ]:
#Obtain values for forecasts of the time series from #year
pred = results.get_prediction(start = pd.to_datetime('1958-01-01'), dynamic = False)
#Obtain associated confidence intervals for forecasts of the time series
pred_ci = pred.conf_int()
pred_ci.head()

Note: The dynamic = False argument ensures that we produce one-step ahead forecasts, meaning that forecasts at each point are generated using the full history up to that point.

In [ ]:
#Plot the forecasted values with historical data
ax = ts_log['1953':].plot(label = 'observed')
pred.predicted_mean.plot(ax = ax, label = 'One-step ahead forecast', alpha = 0.7)
ax.fill_between(pred_ci.index, pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color = 'k', alpha = .2)
ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('1958-01-01'), ts_log.index[-1], alpha = .1, zorder = -1)
plt.xlabel('Year')
plt.ylabel('ts_log')
plt.title('Simple')
plt.legend()

In [ ]:
pred.predicted_mean.head()

In [ ]:
ts_log_forecasted = pred.predicted_mean     #From 1958 to 1960 (validation process)
ts_log_original = ts_log['1958-01-01':]
mse = ((ts_log_forecasted - ts_log_original) ** 2).mean()  #Mean square error
print('Mean Squared Error of forecast : {}'.format(round(mse,3)))

Mean square error is almost zero. This means our prediction is very accurate.

However, a better representation of our true predictive power can be obtained using dynamic forecasts. In this case, we only use information from the time series up to a certain point, and after that, forecasts are generated using values from previous forecasted time points. Let's try with computing the dynamic forecast.

Back to original data

In [ ]:
#Take exponential function
pred_uc = np.exp(pred_uc.predicted_mean)
pred_ci = np.exp(pred_ci)

In [ ]:
#Plot original data prediction
ax = ts['1955':].plot(label='Observed')
pred_uc.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index, pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Year')
ax.set_ylabel('ts (Passengers)')
plt.legend(loc = 'upper left')
plt.show()

Cross validation